Load all the files

In [15]:
import pandas as pd

# Load the CSV files
df_solutions = pd.read_csv("TRAINING_SOLUTIONS.csv")
df_categorical = pd.read_csv("TRAIN_CATEGORICAL_METADATA_new.csv")
df_quantitative = pd.read_csv("TRAIN_QUANTITATIVE_METADATA_new.csv")
df_connectome = pd.read_csv("TRAIN_FUNCTIONAL_CONNECTOME_MATRICES_new_36P_Pearson.csv")

In [26]:
from sklearn.model_selection import train_test_split

#Split the data
train_connectome, test_connectome = train_test_split(df_connectome, test_size=0.2, random_state=42)
train_ids = train_connectome['participant_id']
test_ids = test_connectome['participant_id']
train_connectome.drop('participant_id', axis=1, inplace=True)
test_connectome.drop('participant_id', axis=1, inplace=True)

In [27]:
#Split all the other dataframes along the same bounds
train_categorical = df_categorical[df_categorical['participant_id'].isin(train_ids)]
test_categorical = df_categorical[df_categorical['participant_id'].isin(test_ids)]
train_categorical.drop('participant_id', axis=1, inplace=True)
test_categorical.drop('participant_id', axis=1, inplace=True)

train_quantitative = df_quantitative[df_quantitative['participant_id'].isin(train_ids)]
test_quantitative = df_quantitative[df_quantitative['participant_id'].isin(test_ids)]
train_quantitative.drop('participant_id', axis=1, inplace=True)
test_quantitative.drop('participant_id', axis=1, inplace=True)

train_solutions= df_solutions[df_solutions['participant_id'].isin(train_ids)]
test_solutions = df_solutions[df_solutions['participant_id'].isin(test_ids)]
train_solutions.drop('participant_id', axis=1, inplace=True)
test_solutions.drop('participant_id', axis=1, inplace=True)

<ipython-input-27-1955a826382f>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_categorical.drop('participant_id', axis=1, inplace=True)
<ipython-input-27-1955a826382f>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_categorical.drop('participant_id', axis=1, inplace=True)
<ipython-input-27-1955a826382f>:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_quantitative.drop('participant_id', axis=1, inplace=True)
<ipytho

Preprocess connectome data

In [20]:
train_connectome.head()

,0throw_1thcolumn,0throw_2thcolumn,0throw_3thcolumn,0throw_4thcolumn,0throw_5thcolumn,0throw_6thcolumn,0throw_7thcolumn,0throw_8thcolumn,0throw_9thcolumn,0throw_10thcolumn,...,195throw_196thcolumn,195throw_197thcolumn,195throw_198thcolumn,195throw_199thcolumn,196throw_197thcolumn,196throw_198thcolumn,196throw_199thcolumn,197throw_198thcolumn,197throw_199thcolumn,198throw_199thcolumn
0,0.093473,0.146902,0.067893,0.015141,0.070221,0.063997,0.055382,-0.035335,0.068583,0.029271,...,0.003404,-0.010359,-0.050968,-0.014365,0.128066,0.112646,-0.058980,0.028228,0.133582,0.143372
2,-0.051580,0.139734,0.068295,0.046991,0.111085,0.026978,0.151377,0.021198,0.083721,-0.014275,...,0.053245,-0.028003,0.028773,0.024556,0.166343,0.058925,0.035485,0.063661,0.042862,0.162162
3,0.016273,0.204702,0.115980,0.043103,0.056431,0.057615,0.055773,0.075030,0.001033,-0.064031,...,-0.023918,-0.005356,0.018607,0.016193,0.072955,0.130135,0.056120,0.084784,0.114148,0.190584
4,0.065771,0.098714,0.097604,0.112988,0.071139,0.085607,0.019392,-0.036403,-0.020375,0.005426,...,0.066439,-0.076680,-0.047530,-0.031443,0.221213,0.007343,0.005763,0.083820,0.079582,0.067269
6,0.045489,0.080887,0.073523,0.051792,0.024821,0.083677,0.094708,-0.140014,0.082289,-0.018601,...,-0.000073,-0.062791,0.006491,0.032897,0.072081,-0.016697,-0.069584,0.025494,-0.065204,0.143606


In [28]:
train_connectome.shape

(650, 19900)

In [29]:
train_categorical.shape

(650, 9)

In [30]:
train_solutions.shape

(650, 2)

Train neural network(s?) for connectome data

In [3]:
import tensorflow as tf
from tensorflow.keras import layers, models

# Define model parameters
input_shape = (19900,)
output_shape = 1

# Build the model
def build_model():
    model = models.Sequential()
    model.add(layers.Input(shape=input_shape))

    # Hidden layers
    model.add(layers.Dense(1024, activation='relu'))
    model.add(layers.BatchNormalization())
    model.add(layers.Dropout(0.3))

    model.add(layers.Dense(512, activation='relu'))
    model.add(layers.BatchNormalization())
    model.add(layers.Dropout(0.3))

    model.add(layers.Dense(256, activation='relu'))

    # Output layer
    model.add(layers.Dense(output_shape, activation='softmax'))

    # Compile the model
    model.compile(optimizer='adam',
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])

    return model

# Instantiate the model
nn_adhd = build_model()
nn_f = build_model()


In [ ]:
nn_adhd.fit(train_connectome, train_solutions['ADHD_Outcome'], epochs=10, batch_size=32, validation_split=0.2)

In [ ]:
nn_f.fit(train_connectome, train_solutions['Sex_F'], epochs=10, batch_size=32, validation_split=0.2)

In [ ]:
# Get predictions and confidence scores
# Assuming `X_test` is your test dataset of shape (650, 19900)
predictions = model.predict(test_connectome)

# Each row in `predictions` corresponds to the class probabilities
for i, pred in enumerate(predictions):
    print(f"Sample {i}: Class 0 Confidence: {pred[0]:.4f}, Class 1 Confidence: {pred[1]:.4f}")


Preprocess categorical data

In [41]:
train_quantitative.head()

,EHQ_EHQ_Total,ColorVision_CV_Score,APQ_P_APQ_P_CP,APQ_P_APQ_P_ID,APQ_P_APQ_P_INV,APQ_P_APQ_P_OPD,APQ_P_APQ_P_PM,APQ_P_APQ_P_PP,SDQ_SDQ_Conduct_Problems,SDQ_SDQ_Difficulties_Total,SDQ_SDQ_Emotional_Problems,SDQ_SDQ_Externalizing,SDQ_SDQ_Generating_Impact,SDQ_SDQ_Hyperactivity,SDQ_SDQ_Internalizing,SDQ_SDQ_Peer_Problems,SDQ_SDQ_Prosocial,MRI_Track_Age_at_Scan
1,-94.47,14,3,13,34,18,23,30,0,18,6,8,7,8,10,4,5,NaN
4,0.00,14,5,15,40,20,24,28,1,18,4,11,4,10,7,3,9,8.940679
6,94.47,14,3,14,27,18,20,21,0,9,1,5,1,5,4,3,10,16.768195
7,60.00,14,3,15,41,21,19,25,3,16,3,11,6,8,5,2,6,11.221309
10,31.13,14,3,16,43,18,31,27,1,14,3,6,6,5,8,5,5,11.812457


In [42]:
from sklearn.preprocessing import MinMaxScaler
import pandas as pd

# Function to fill NaNs with mean and scale between 0 and 1
def preprocess_dataframe(df):
    # Fill NaN values with column mean
    df_filled = df.fillna(df.mean())

    # Scale values between 0 and 1
    scaler = MinMaxScaler()
    df_scaled = pd.DataFrame(scaler.fit_transform(df_filled), columns=df.columns, index=df.index)

    return df_scaled

# Apply to train_quantitative and test_quantitative
train_quantitative = preprocess_dataframe(train_quantitative)
test_quantitative = preprocess_dataframe(test_quantitative)


Train LightGBM on categorical

In [45]:
import lightgbm as lgb
import pandas as pd
import numpy as np

# Ensure train_quantitative and train_solutions are properly aligned
assert len(train_quantitative) == len(train_solutions), "Mismatch in data dimensions"

# Prepare LightGBM dataset
train_adhd = lgb.Dataset(train_quantitative, label=train_solutions['ADHD_Outcome'])
train_f = lgb.Dataset(train_quantitative, label=train_solutions['Sex_F'])

# LightGBM parameters (for binary classification)
params = {
    'objective': 'binary',          # Binary classification
    'metric': 'binary_logloss',     # Log loss metric
    'boosting_type': 'gbdt',        # Gradient boosting decision tree
    'num_leaves': 31,               # Maximum number of leaves
    'learning_rate': 0.05,          # Step size shrinkage
    'verbose': -1                   # Suppress output
}

# Train the LightGBM model
lgb_adhd = lgb.train(params, train_adhd, num_boost_round=100)
lgb_f = lgb.train(params, train_f, num_boost_round=100)

In [ ]:
# Predict probabilities on test data
test_probabilities_adhd = lgb_adhd.predict(test_quantitative)

# Display test probabilities
for i, prob in enumerate(test_probabilities_adhd):
    print(f"Test Sample {i}: Probability of positive class: {prob:.4f}")


In [ ]:
# Predict probabilities on test data
test_probabilities_f = lgb_f.predict(test_quantitative)

# Display test probabilities
for i, prob in enumerate(test_probabilities_f):
    print(f"Test Sample {i}: Probability of positive class: {prob:.4f}")


In [ ]:
def final_output(model_output, model_confidence, lgb_output, lgb_confidence):
  output = model_output * model_confidence + lgb_output * lgb_confidence
  if output >= 0.5:
    return 1
  else:
    return 0